### Demonstration Overview: Constructing the `Variation` Class

This notebook demonstrates the construction of the `Variation` class. The `Variation` class, built with Pydantic, extends the `MolecularDefinition` class to meet the specific requirements of the HL7 FHIR MolecularDefinition Variation Profile. It incorporates tailored attributes and behaviors to support this use case. This notebook demonstrates how to use this class effectively, and shows how to build an Variation example step by step.


### Step 1: Prerequisites and Setup

To construct the `Variation` class in this demonstration, we set up the environment by importing the specific libraries and modules required for the examples in this notebook. These include:

1. **External Package**:
   - `Coding`, `CodeableConcept`, `Quantity`, `Reference`, and `Meta` from the `fhir.resources` library. These classes provide the framework for working with structured healthcare data in FHIR format.

2. **Custom Project Modules**:
   - `Variation` from `profiles.variation`: The main class we will build and demonstrate.
   - Components from `resources.moleculardefinition`:
     - `MolecularDefinitionLocation` and related classes for specifying molecular location.
     - `MolecularDefinitionRepresentation` and related classes for specifying  molecular representations.

By importing these libraries, we ensure access to all the tools needed for this demonstration.


In [1]:
# Import required libraries
from fhir.resources.coding import Coding
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.quantity import Quantity
from fhir.resources.reference import Reference
from fhir.resources.meta import Meta
from profiles.variation import Variation as FhirVariation

from resources.moleculardefinition import (
    MolecularDefinitionLocation,
    MolecularDefinitionLocationSequenceLocation,
    MolecularDefinitionLocationSequenceLocationCoordinateInterval,
    MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem,
    MolecularDefinitionRepresentation,
    MolecularDefinitionRepresentationLiteral
)


### Step 2: Defining the Variation Location Component

In this step, we define the location component for the `Variation`, which is a subclass of the `MolecularDefinition`. 

- One necessary modification was adjusting the cardinality of the `location` attribute from `0..*` to `1..1` to ensure a mandatory and singular location specification.

#### Example Workflow:
The JSON location is implemented in Python as follows in next cell:
```json
{
  "location": [
    {
    "sequenceLocation" : {
      "sequenceContext" : {
        "reference" : "MolecularDefinition/example-sequence-nm0007694-url",
        "type" : "MolecularDefinition",
        "display" : "Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4"
      },
      "coordinateInterval" : {
        "coordinateSystem" : {
          "system" : {
            "coding" : [{
              "system" : "http://loinc.org",
              "code" : "LA30100-4",
              "display" : "0-based interval counting"
            }]
          }
        },
        "startQuantity" : {
          "value" : 1015
        }
      }
    }
  }
  ]
}

In [2]:
# Define the coding for the coordinateSystem
coding_val = Coding(
    system="http://loinc.org",
    code="LA30100-4",
    display="0-based interval counting",
)

# Wrap coding into the  CodeableConcept object
codeconcept_val = CodeableConcept(
    coding=[coding_val],
    text="0-based interval counting",
)

# Wrap the CodeableConcept into coordinateSystem for location
MolDefLocSeqLocCoordIntCoord = MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem(
    system=codeconcept_val
)
# Create the coordinateInterval and build the interval with start and end quantities.
MolDefLocSeqLocCoordInt = MolecularDefinitionLocationSequenceLocationCoordinateInterval(
    coordinateSystem=MolDefLocSeqLocCoordIntCoord,
    startQuantity=Quantity(value=1015),
)
# Wrap the coordianteInterval into the sequenceLocation and add the sequenceContext.
MolDefLocSeqLoc = MolecularDefinitionLocationSequenceLocation(
    sequenceContext=Reference(
        reference="MolecularDefinition/example-sequence-nm0007694-url",
        type="MolecularDefinition",
        display="Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4",
    ),
    coordinateInterval=MolDefLocSeqLocCoordInt,
)
# Create the MolecularDefinitionLocation object, including the sequenceLocation
MolDefLoc = MolecularDefinitionLocation(
    sequenceLocation=MolDefLocSeqLoc
)

# Inspect serialized output
MolDefLoc.model_dump()


{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nm0007694-url',
   'type': 'MolecularDefinition',
   'display': 'Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4'},
  'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
       'code': 'LA30100-4',
       'display': '0-based interval counting'}],
     'text': '0-based interval counting'}},
   'startQuantity': {'value': 1015.0}}}}

### Step 3: Defining the Variation Representation Component

In this step, we define the representation component for the `Variation`, which is a subclass of the `MolecularDefinition`. 

- One necessary modification was adjusting the cardinality of the `representation` attribute from `0..*` to `1..1` to ensure a mandatory and singular representation specification.  
#TODO: EDIT add the fact the Variation requires Reference State and Alternative State

#### JSON Representation:
```json
{
  "representation": [
    {
    "focus" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        "code" : "reference-state",
        "display" : "Reference State"
      }]
    },
    "literal" : {
      "value" : "A"
    }
  },
  {
    "focus" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        "code" : "alternative-state",
        "display" : "Alternative State"
      }]
    },
    "literal" : {
      "value" : "G"
    }
  }
  ]
}


In [3]:
# Create a Coding instance to specify the focus.  
reference_state_val = Coding(
        system="http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        code= "reference-state",
        display="Reference State"
    )
# Wrapt the coding value into the CodeableConcept.
reference_state_focus = CodeableConcept(
    coding=[reference_state_val],
)

#Define the literal value using MolecularDefinitionRepresentationLiteral.
MolDefRepLit = MolecularDefinitionRepresentationLiteral(value="A")

# Integrate these into the MolecularDefinitionRepresentation. 
MolDefRepRefState = MolecularDefinitionRepresentation(
    literal=MolDefRepLit,
    focus=reference_state_focus
    )

# Follow similar steps as above, but for Alternative State 

alternative_state_val = Coding(
        system="http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        code= "alternative-state",
        display="Alternative State"
    )

alternative_state_focus_val = CodeableConcept(
    coding=[alternative_state_val],
)

MolDefRepLitAltState = MolecularDefinitionRepresentationLiteral(value="G")

MolDefRepAltState = MolecularDefinitionRepresentation(
    literal=MolDefRepLitAltState,
    focus=alternative_state_focus_val
    )

MolDefRepRefState.model_dump(),MolDefRepAltState.model_dump()


({'focus': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus',
     'code': 'reference-state',
     'display': 'Reference State'}]},
  'literal': {'value': 'A'}},
 {'focus': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus',
     'code': 'alternative-state',
     'display': 'Alternative State'}]},
  'literal': {'value': 'G'}})

### Step 4: Integrating Components into the Complete `Variation`

In this step, we combine all the previously defined components to create a complete instance of the `Variation` class. This integration includes meta data including (resourceType, id, meta, moleculeType), location and representation components.

- One necessary modification was adjusting the cardinality of the `moleculeType` attribute from `0..1` to `1..1` to ensure a mandatory and singular specification.  
- Additionally, the `memberState` attribute has been removed as it is not required in the `Variation`.  
 
#### JSON Representation:
```json
{
  "resourceType": "MolecularDefinition",
  "id": "example-variation-cyp2c19-1015-spdi",
  "meta": {
    "profile": ["http://hl7.org/fhir/uv/molecular-definition-data-types/StructureDefinition/variation"]
  },
  "moleculeType": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecule-type",
        "code": "dna",
        "display": "DNA Sequence"
      }
    ]
  },
  "location": [
    {...}  // Details from MolecularDefinitionLocation
  ],
  "representation": [
    {...}  // Details from MolecularDefinitionRepresentation
  ]
}


In [4]:
# Spcifies the profile URL in the Meta data type
meta_val = Meta(
  profile= ["http://hl7.org/fhir/uv/molecular-definition-data-types/StructureDefinition/variation"]
)

# Defines the type of sequence, such as dna, rna, or aa.
moltype_coding_val = Coding(
        system="http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecule-type",
        code= "dna",
        display= "DNA Sequence"
    )

# Wrap the moleculeType in the CodeableConcept data type.
moltype = CodeableConcept(
    coding=[moltype_coding_val],
)
#Incorporates the Location and Representation components created earlier.
Allele_Example = FhirVariation(
  id = 'demo-example-hgvs-substitution',
  meta = meta_val,
  moleculeType=moltype,
  location=[MolDefLoc],
  representation=[MolDefRepRefState,MolDefRepAltState]
)

# Inspect serialized output
Allele_Example.model_dump()


{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitution',
 'meta': {'profile': ['http://hl7.org/fhir/uv/molecular-definition-data-types/StructureDefinition/variation']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecule-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nm0007694-url',
     'type': 'MolecularDefinition',
     'display': 'Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}],
       'text': '0-based interval counting'}},
     'startQuantity': {'value': 1015.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/mo

### Final Example: Using JSON to Generate the Class

Constructing the `Variation` step by step can be a bit challenging, especially if you are new to the `MolecularDefinition` schema. To ensure accuracy while constructing the `Variation`, we recommend having the schema available as a reference for guidance. 

Alternatively, instead of constructing the `Variation` step by step, you can use Python's `**` unpacking syntax to directly generate the class instance from the complete JSON structure. This method simplifies the process and ensures the class instance accurately reflects the provided JSON data.


In [5]:
example_spdi = {
  "resourceType" : "MolecularDefinition",
  "id" : "example-variation-cyp2c19-1015-spdi",
  "meta" : {
    "profile" : ["http://hl7.org/fhir/uv/molecular-definition-data-types/StructureDefinition/variation"]
  },
  "moleculeType" : {
    "coding" : [{
      "system" : "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecule-type",
      "code" : "dna",
      "display" : "DNA Sequence"
    }]
  },
  "location" : [{
    "sequenceLocation" : {
      "sequenceContext" : {
        "reference" : "MolecularDefinition/example-sequence-nm0007694-url",
        "type" : "MolecularDefinition",
        "display" : "Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4"
      },
      "coordinateInterval" : {
        "coordinateSystem" : {
          "system" : {
            "coding" : [{
              "system" : "http://loinc.org",
              "code" : "LA30100-4",
              "display" : "0-based interval counting"
            }]
          }
        },
        "startQuantity" : {
          "value" : 1015
        }
      }
    }
  }],
  "representation" : [{
    "focus" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        "code" : "reference-state",
        "display" : "Reference State"
      }]
    },
    "literal" : {
      "value" : "A"
    }
  },
  {
    "focus" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus",
        "code" : "alternative-state",
        "display" : "Alternative State"
      }]
    },
    "literal" : {
      "value" : "G"
    }
  }]
}

FhirVariation(**example_spdi).model_dump()

{'resourceType': 'MolecularDefinition',
 'id': 'example-variation-cyp2c19-1015-spdi',
 'meta': {'profile': ['http://hl7.org/fhir/uv/molecular-definition-data-types/StructureDefinition/variation']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecule-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nm0007694-url',
     'type': 'MolecularDefinition',
     'display': 'Starting Sequence Resource: (CYP2C19), mRNA, NM_000769.4'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}]}},
     'startQuantity': {'value': 1015.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/uv/molecular-definition-data-types/CodeSystem/molecular-definition-focus',
      'code'

### Conclusion

The `Variation` resource is currently under development. For more details and the latest updates, please refer to the [HL7 FHIR Variation Documentation](https://build.fhir.org/ig/HL7/molecular-definition-data-types/StructureDefinition-variation.html).

**Note:** The documentation may occasionally experience downtime, which is beyond the control of our lab group. We appreciate your patience and apologize for any inconvenience this may cause.